In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

def clean_text(text):
    return ' '.join(text.split())  # Hapus newline dan gabungkan teks menjadi satu baris

# Inisialisasi driver Chrome
driver = webdriver.Chrome()
driver.set_window_size(1300, 800)

# Menentukan jumlah halaman yang ingin diambil
num_pages = 1  # Ubah sesuai kebutuhan Anda

questions_data = []

# Loop untuk setiap halaman
for page_num in range(1, num_pages + 1):
    # URL halaman dengan nomor halaman dinamis
    link = f"https://stackoverflow.com/questions/tagged/python?tab=newest&page={page_num}&pagesize=50"
    driver.get(link)
    
    # Ambil source page HTML
    content = driver.page_source
    
    # Parse konten HTML dengan BeautifulSoup
    data = BeautifulSoup(content, 'html.parser')
    
    # Temukan semua elemen judul yang memiliki link ke pertanyaan
    for area in data.find_all('div', class_="s-post-summary js-post-summary"):
        title = area.find('a', class_="s-link").get_text(strip=True)
        question_excerpt = clean_text(area.find('div', class_="s-post-summary--content-excerpt").get_text(strip=True))
        votes = area.select_one('.s-post-summary--stats-item__emphasized .s-post-summary--stats-item-number').get_text(strip=True)
        answers = area.select_one('.s-post-summary--stats-item:nth-child(2) .s-post-summary--stats-item-number').get_text(strip=True)
        views = area.select_one('.s-post-summary--stats-item:nth-child(3) .s-post-summary--stats-item-number').get_text(strip=True)
        tags = [tag.get_text(strip=True) for tag in area.select('.s-post-summary--meta-tags .post-tag')]
        author = area.select_one('.s-user-card--info .flex--item').get_text(strip=True) if area.select_one('.s-user-card--info .flex--item') else 'Anonymous'
        reputation = area.select_one('.s-user-card--rep').get_text(strip=True) if area.select_one('.s-user-card--rep') else '0'
        link = "https://stackoverflow.com" + area.find('a', class_="s-link")['href']

        # Simpan data ke dalam dictionary
        question_info = {
            'Title': title,
            'Excerpt': question_excerpt,
            'Votes': votes,
            'Answers': answers,
            'Views': views,
            'Tags': ', '.join(tags),
            'Author': author,
            'Reputation': reputation,
            'Link': link
        }

        questions_data.append(question_info)

# Tutup driver setelah selesai
driver.quit()
# Simpan DataFrame ke file CSV

output_file = 'HasilScrapping_Selenium.csv'
df_questions = pd.DataFrame(questions_data)
df_questions.to_csv(output_file, index=False)